# (Ford-GoBike Share)
In this notebook, we are going to use 'Datawig', a deeplearnig library specifically used for missing values imputation. There are two main features to impute 'Age' and 'Gender'.
But first we will carry out some data manipulation and wrangling to compute the desired features to be used as input features in the learning algorithm.   

## Preliminary Wrangling

> This data set includes information about individual rides made in a bike-sharing system covering the greater San Francisco Bay area. Time and dates at which riders used to go on a ride, also some members identifications like age and membership status are included as well for each member. To analyze the main factors upon which may or may not affect the duration the individual take during the trip.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from math import radians, cos, sin, asin, sqrt

In [3]:
df = pd.read_csv('2019-fordgobike-San Franscisco.csv')
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,52185,2019-02-28 17:32:10.1450,2019-03-01 08:01:55.9750,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,13.0,Commercial St at Montgomery St,37.794231,-122.402923,4902,Customer,1984.0,Male,No
1,42521,2019-02-28 18:53:21.7890,2019-03-01 06:42:03.0560,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,81.0,Berry St at 4th St,37.775880,-122.393170,2535,Customer,NaN,NaN,No
2,61854,2019-02-28 12:13:13.2180,2019-03-01 05:24:08.1460,86.0,Market St at Dolores St,37.769305,-122.426826,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,5905,Customer,1972.0,Male,No
3,36490,2019-02-28 17:54:26.0100,2019-03-01 04:02:36.8420,375.0,Grove St at Masonic Ave,37.774836,-122.446546,70.0,Central Ave at Fell St,37.773311,-122.444293,6638,Subscriber,1989.0,Other,No
4,1585,2019-02-28 23:54:18.5490,2019-03-01 00:20:44.0740,7.0,Frank H Ogawa Plaza,37.804562,-122.271738,222.0,10th Ave at E 15th St,37.792714,-122.248780,4898,Subscriber,1974.0,Male,Yes


In [4]:
print(df.shape)
df.info()

(183412, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183412 entries, 0 to 183411
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             183412 non-null  int64  
 1   start_time               183412 non-null  object 
 2   end_time                 183412 non-null  object 
 3   start_station_id         183215 non-null  float64
 4   start_station_name       183215 non-null  object 
 5   start_station_latitude   183412 non-null  float64
 6   start_station_longitude  183412 non-null  float64
 7   end_station_id           183215 non-null  float64
 8   end_station_name         183215 non-null  object 
 9   end_station_latitude     183412 non-null  float64
 10  end_station_longitude    183412 non-null  float64
 11  bike_id                  183412 non-null  int64  
 12  user_type                183412 non-null  object 
 13  member_birth_year        175147 non-null  floa

In [5]:
(df['member_birth_year'].isnull()==df['member_gender'].isnull()).count()

183412

This indicates that all the null values for member_birth_year and member_gender colums are on the same rows.

In [6]:
df.describe()

,duration_sec,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bike_id,member_birth_year
count,183412.000000,183215.000000,183412.000000,183412.000000,183215.000000,183412.000000,183412.000000,183412.000000,175147.000000
mean,726.078435,138.590427,37.771223,-122.352664,136.249123,37.771427,-122.352250,4472.906375,1984.806437
std,1794.389780,111.778864,0.099581,0.117097,111.515131,0.099490,0.116673,1664.383394,10.116689
min,61.000000,3.000000,37.317298,-122.453704,3.000000,37.317298,-122.453704,11.000000,1878.000000
25%,325.000000,47.000000,37.770083,-122.412408,44.000000,37.770407,-122.411726,3777.000000,1980.000000
50%,514.000000,104.000000,37.780760,-122.398285,100.000000,37.781010,-122.398279,4958.000000,1987.000000
75%,796.000000,239.000000,37.797280,-122.286533,235.000000,37.797320,-122.288045,5502.000000,1992.000000
max,85444.000000,398.000000,37.880222,-121.874119,398.000000,37.880222,-121.874119,6645.000000,2001.000000


In [7]:
df[df['member_birth_year']== 1878]

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
27370,1474,2019-02-25 09:28:42.1610,2019-02-25 09:53:17.0680,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,386.0,24th St at Bartlett St,37.752105,-122.419724,6436,Customer,1878.0,Female,No


In [8]:
df.drop(df[df['member_birth_year']== 1878].index,inplace=True)

In [9]:
print('Total number of stations :' , df['start_station_name'].value_counts().count())
print('Total number of bikes :', df['bike_id'].value_counts().count())

Total number of stations : 329
Total number of bikes : 4646


There is a total of 4646 bikes and 329 stations.

In [10]:
df['bike_id']=df['bike_id'].apply(str)

In [11]:
print('Total number of stations :' , df['start_station_name'].value_counts().count())
print('Total number of bikes :', df['bike_id'].value_counts().count())

Total number of stations : 329
Total number of bikes : 4646


- After deleting the rows with missing values. There is 4606 bikes present in the dataset.<br> This means there is a total of 40 bikes id deleted, this is less than 1% of the total bikes so this should not really affect the analysis.
- The total number of stations is still the same.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183411 entries, 0 to 183411
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             183411 non-null  int64  
 1   start_time               183411 non-null  object 
 2   end_time                 183411 non-null  object 
 3   start_station_id         183214 non-null  float64
 4   start_station_name       183214 non-null  object 
 5   start_station_latitude   183411 non-null  float64
 6   start_station_longitude  183411 non-null  float64
 7   end_station_id           183214 non-null  float64
 8   end_station_name         183214 non-null  object 
 9   end_station_latitude     183411 non-null  float64
 10  end_station_longitude    183411 non-null  float64
 11  bike_id                  183411 non-null  object 
 12  user_type                183411 non-null  object 
 13  member_birth_year        175146 non-null  float64
 14  memb

**After Some exploration to the data we going to remove the unwanted columns**

In [13]:
#we want to find the disatance between the start_station and end_station for each trip.

lat_1 = df['start_station_latitude']/(180/np.pi)
lat_2 = df['end_station_latitude']/(180/np.pi)
lon_1 = df['start_station_longitude']/(180/np.pi)
lon_2 = df['end_station_longitude']/(180/np.pi)

dlon = lon_2 - lon_1
dlat = lat_2 - lat_1

a = np.sin(dlat / 2)**2 + np.cos(lat_1) * np.cos(lat_2) * np.sin(dlon / 2)**2
c = 2 * (np.arcsin(np.sqrt(a)))
r= 6371
df['distance/Km'] = c*6371

In [14]:
df.drop(columns=['start_station_id','start_station_latitude','start_station_longitude','end_station_latitude','end_station_longitude','end_station_id'],inplace=True)

In [15]:
df['distance/Km']=df['distance/Km'].round(decimals=2)
df.head()

,duration_sec,start_time,end_time,start_station_name,end_station_name,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,distance/Km
0,52185,2019-02-28 17:32:10.1450,2019-03-01 08:01:55.9750,Montgomery St BART Station (Market St at 2nd St),Commercial St at Montgomery St,4902,Customer,1984.0,Male,No,0.54
1,42521,2019-02-28 18:53:21.7890,2019-03-01 06:42:03.0560,The Embarcadero at Steuart St,Berry St at 4th St,2535,Customer,NaN,NaN,No,1.74
2,61854,2019-02-28 12:13:13.2180,2019-03-01 05:24:08.1460,Market St at Dolores St,Powell St BART Station (Market St at 4th St),5905,Customer,1972.0,Male,No,2.70
3,36490,2019-02-28 17:54:26.0100,2019-03-01 04:02:36.8420,Grove St at Masonic Ave,Central Ave at Fell St,6638,Subscriber,1989.0,Other,No,0.26
4,1585,2019-02-28 23:54:18.5490,2019-03-01 00:20:44.0740,Frank H Ogawa Plaza,10th Ave at E 15th St,4898,Subscriber,1974.0,Male,Yes,2.41


In [16]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

In [17]:
df['start_day']= df['start_time'].dt.day_name()
df['start_hour']= df['start_time'].dt.hour


In [18]:
df['duration'] = pd.to_timedelta(df['duration_sec'], unit='s')
df.head()

,duration_sec,start_time,end_time,start_station_name,end_station_name,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,distance/Km,start_day,start_hour,duration
0,52185,2019-02-28 17:32:10.145,2019-03-01 08:01:55.975,Montgomery St BART Station (Market St at 2nd St),Commercial St at Montgomery St,4902,Customer,1984.0,Male,No,0.54,Thursday,17,0 days 14:29:45
1,42521,2019-02-28 18:53:21.789,2019-03-01 06:42:03.056,The Embarcadero at Steuart St,Berry St at 4th St,2535,Customer,NaN,NaN,No,1.74,Thursday,18,0 days 11:48:41
2,61854,2019-02-28 12:13:13.218,2019-03-01 05:24:08.146,Market St at Dolores St,Powell St BART Station (Market St at 4th St),5905,Customer,1972.0,Male,No,2.70,Thursday,12,0 days 17:10:54
3,36490,2019-02-28 17:54:26.010,2019-03-01 04:02:36.842,Grove St at Masonic Ave,Central Ave at Fell St,6638,Subscriber,1989.0,Other,No,0.26,Thursday,17,0 days 10:08:10
4,1585,2019-02-28 23:54:18.549,2019-03-01 00:20:44.074,Frank H Ogawa Plaza,10th Ave at E 15th St,4898,Subscriber,1974.0,Male,Yes,2.41,Thursday,23,0 days 00:26:25


In [19]:
df['age']= df.start_time.dt.year - df.member_birth_year

In [26]:
#Replace the position of the columns in the dataframe to look more organized.
df.insert(0,'duration', df.pop('duration'))
df.insert(9,'age', df.pop('age'))
df.insert(3,'start_day',df.pop('start_day'))
df.insert(4,'start_hour',df.pop('start_hour'))


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183411 entries, 0 to 183411
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype          
---  ------                   --------------   -----          
 0   duration                 183411 non-null  timedelta64[ns]
 1   duration_sec             183411 non-null  int64          
 2   start_time               183411 non-null  datetime64[ns] 
 3   start_day                183411 non-null  object         
 4   start_hour               183411 non-null  int64          
 5   end_time                 183411 non-null  datetime64[ns] 
 6   start_station_name       183214 non-null  object         
 7   end_station_name         183214 non-null  object         
 8   bike_id                  183411 non-null  object         
 9   user_type                183411 non-null  object         
 10  member_birth_year        175146 non-null  float64        
 11  age                      175146 non-null  float64        
 12  me

In [22]:
 df.drop(df[df['start_station_name'].isnull()].index, inplace=True)

In [23]:
df.isna().sum()

duration                      0
duration_sec                  0
start_time                    0
start_day                     0
start_hour                    0
end_time                      0
start_station_name            0
end_station_name              0
bike_id                       0
user_type                     0
member_birth_year          8263
age                        8263
member_gender              8263
bike_share_for_all_trip       0
distance/Km                   0
dtype: int64

We deleted the rows with missing values on the start_station_name and end_station_name as they are only 197 rows and would not affect the analysis.<br> Now there are 8263 rows with missing values of age and membr_gender. We will predict the missing values using datawig.

In [37]:
df.corr()

,duration_sec,start_hour,age,member_birth_year,distance/Km
duration_sec,1.000000,0.006640,0.006015,-0.006015,0.126898
start_hour,0.006640,1.000000,-0.064089,0.064089,-0.068644
age,0.006015,-0.064089,1.000000,-1.000000,0.020084
member_birth_year,-0.006015,0.064089,-1.000000,1.000000,-0.020084
distance/Km,0.126898,-0.068644,0.020084,-0.020084,1.000000


From the correlation table shown above, we can see that there is no strong correlation between any of the numeric features.

In [31]:
df_pred=df[df['member_gender'].isnull()]

## Datawig for imputation
We will use datawig library which is a framework used impute missing values in tables.<br>
Datawig is a Deep Learning library developed by AWS Labs and is primarily used for missing values imputation.<br>
DataWig currently works with only python version 3.7 and below.

In [47]:
from datawig.utils import random_split
from datawig import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error



In [66]:
# Split the rows containing no null values into training and test data.
df_train, df_test = random_split(df.dropna(), split_ratios=[0.8, 0.2])
input_cols = ['duration_sec','start_day','start_hour', 'start_station_name','distance/Km','user_type']
output_age = 'age' # Numerical output
output_gender = 'member_gender' # Categorical output

In [39]:
# Initialize imputer for numerical imputation
imputer_age = SimpleImputer(
                 input_columns=input_cols,
                 output_column=output_age,  # Column to be imputed
                 output_path='imputer_age'  # Store model, data, and metrics
                 )

# Initialize imputer for categorical imputation
imputer_gender = SimpleImputer(
                 input_columns=input_cols,
                 output_column=output_gender,  # Column to be imputed
                 output_path='imputer_gender'  # Store model, data, and metrics
                 )

In [40]:
# Imputation modbel fit
imputer_age.fit(train_df=df_train)
imputer_gender.fit(train_df=df_train) 

2023-01-05 08:51:45,936 [INFO]  
========== start: fit model
2023-01-05 08:51:45,959 [WARNING]  Already bound, ignoring bind()
2023-01-05 08:52:16,918 [INFO]  Epoch[0] Batch [0-3937]	Speed: 2038.12 samples/sec	cross-entropy=15.228501	age-accuracy=0.000000
2023-01-05 08:52:48,208 [INFO]  Epoch[0] Train-cross-entropy=15.022992
2023-01-05 08:52:48,209 [INFO]  Epoch[0] Train-age-accuracy=0.000000
2023-01-05 08:52:48,210 [INFO]  Epoch[0] Time cost=62.236
2023-01-05 08:52:48,623 [INFO]  Saved checkpoint to "imputer_age\model-0000.params"
2023-01-05 08:52:53,583 [INFO]  Epoch[0] Validation-cross-entropy=14.643365
2023-01-05 08:52:53,586 [INFO]  Epoch[0] Validation-age-accuracy=0.000000
2023-01-05 08:53:25,924 [INFO]  Epoch[1] Batch [0-3937]	Speed: 1948.42 samples/sec	cross-entropy=14.671208	age-accuracy=0.000000
2023-01-05 08:53:58,991 [INFO]  Epoch[1] Train-cross-entropy=14.696787
2023-01-05 08:53:58,992 [INFO]  Epoch[1] Train-age-accuracy=0.000000
2023-01-05 08:53:58,994 [INFO]  Epoch[1] Ti

In [68]:
# Predict the age and gender for the test data.
predictions_age = imputer_age.predict(df_test)
predictions_gender = imputer_gender.predict(df_test)


In [63]:
predictions_age.head()

,duration,duration_sec,start_time,start_day,start_hour,end_time,start_station_name,end_station_name,bike_id,user_type,member_birth_year,age,member_gender,bike_share_for_all_trip,distance/Km,age_imputed
113095,00:07:08,428,2019-02-12 13:12:48.871,Tuesday,13,2019-02-12 13:19:57.023,Steuart St at Market St,Powell St BART Station (Market St at 4th St),4979,Subscriber,1957.0,62.0,Male,No,1.26,36.922214
173295,00:17:02,1022,2019-02-03 12:08:36.392,Sunday,12,2019-02-03 12:25:38.464,Folsom St at 3rd St,Market St at 10th St,5506,Subscriber,1989.0,30.0,Male,No,1.81,33.074591
33434,00:04:46,286,2019-02-24 11:48:59.229,Sunday,11,2019-02-24 11:53:45.516,The Embarcadero at Sansome St,Broadway at Battery St,1267,Subscriber,1963.0,56.0,Female,No,0.72,34.101057
13872,00:08:14,494,2019-02-27 14:26:36.627,Wednesday,14,2019-02-27 14:34:50.782,Townsend St at 7th St,1st St at Folsom St,5459,Subscriber,1988.0,31.0,Male,No,1.95,33.176128
22112,00:10:31,631,2019-02-26 08:29:03.948,Tuesday,8,2019-02-26 08:39:35.584,San Francisco Caltrain Station 2 (Townsend St...,Howard St at Beale St,4337,Subscriber,1995.0,24.0,Male,No,1.46,35.143890


In [59]:
print(predictions_gender.shape)
print("Value counts of predicted results:\n",predictions_gender.member_gender_imputed.value_counts())
print("Value counts of actual results:\n",predictions_gender.member_gender.value_counts())

(34990, 17)
Value counts of predicted results:
 Male      34835
Female      155
Name: member_gender_imputed, dtype: int64
Value counts of actual results:
 Male      26083
Female     8151
Other       756
Name: member_gender, dtype: int64


99.5% of the predicted values are males. However, the actual test data set has only 65.8 % males.<br> This shows that the model is very unrelaiable for predicting the gender.<br>
This can be because there is no strong correlation between features to identify the gender  alot of outliers in most of the features of the dataset.

In [62]:
mse_age = mean_absolute_error(df_test[output_age],
                                 predictions_age[f'{output_age}_imputed'])

# Evaluation of categorical imputation
mcc_gender = matthews_corrcoef(df_test[output_gender],
                                predictions_gender[f'{output_gender}_imputed'])

print('The mean absolute error for the age predicted: {}\n The matthews correlation coefficient for the predicted gender: {}'.format(mse_age,mcc_gender))

The mean absolute error for the age predicted: 7.0663095767147395
 The matthews correlation coefficient for the predicted gender: 0.045066254594691345


For the age predicted, there is almost 7 years mean error for each imputation.<br>
For the gender predicted, the matthews correlation is very close to 0 which indicates that the prediction is very close to random.

## Conclusion
After testing the predicted results, we found out that the model is not reliable and accurate enough to give us the required predicitions.<br>
As mentioned, this high inaccuracy can be due to the foact that the features of the dataset have no strong correlatins to each other and have high number of outliers.<br>
In this case, using a machine learning model to predict the missing values ('Gender' specifically ) is not the best option since there are no features that strongly correlate to each other which can be used to specify the gender of the user.